# TEXT2SQL with transformers

In [1]:
import torch
import transformers
print(f"PyTroch Version: {torch.__version__}")
print(f"Transfomers Version: {transformers.__version__}")

PyTroch Version: 1.7.1
Transfomers Version: 4.6.1


In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("monologg/kobert")
model.embeddings.word_embeddings